In [3]:
import torch
#Create the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print("Executing the model on :",device)

Executing the model on : cuda:0


In [4]:
import pandas as pd
import numpy as np
import os

ex_data = pd.read_csv(os.path.join('..', 'data', 'ex_data.csv'))

ex_data['admin_date'] = ex_data['admin_date'].astype(int).astype(str)
in_date = pd.to_datetime(ex_data['admin_date'], format='%Y/%m/%d', errors='coerce')

ex_data['discharge_date'] = ex_data['discharge_date'].astype(int).astype(str)
out_date = pd.to_datetime(ex_data['discharge_date'], format='%Y/%m/%d', errors='coerce')

day_diff = out_date - in_date
ex_data['duration'] = day_diff.dt.days

y_data = ex_data[['SurvivalWeeks']]
X_data = ex_data.drop(['ID', 'CHT_NO', 'admin_date', 'discharge_date',
                       'AllMortality', 'CVDeath  ', 'Death Date', 'SurvivalWeeks'], axis=1)

categorical_columns = ['Sex', 'AF', 'DM', 'HTN', 'Dyslipidemia', 'CHF', 'Smoking', 'Cancer before adm']
numerical_columns =  np.setdiff1d(X_data.columns, categorical_columns)

# Basic NN

In [5]:
from torch import nn
import torch.utils.data as Data
import torch.nn.functional as F

class DNN (nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(30, 15)
        self.fc2 = nn.Linear(15, 7)
        self.fc3 = nn.Linear(7, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

model = DNN()
print(model)

DNN(
  (fc1): Linear(in_features=30, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=7, bias=True)
  (fc3): Linear(in_features=7, out_features=1, bias=True)
)


In [6]:
# Settings
epochs = 10
batch_size = 128
lr = 0.008

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = nn.MSELoss()

In [7]:
# for epoch in range(epochs):


SyntaxError: unexpected EOF while parsing (<ipython-input-7-f7734f86eb01>, line 3)

In [ ]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from imblearn import over_sampling
from sklearn.metrics import auc, roc_curve

# one-hot
X_data_one_hot = pd.get_dummies(X_data, columns=categorical_columns)
y_data_od = (y_data<24).astype(int)

all_auroc = []
for train_index, test_index in KFold(n_splits=10, random_state=42, shuffle=True).split(X_data_one_hot):
    X_train, X_test = X_data_one_hot.iloc[train_index], X_data_one_hot.iloc[test_index]
    y_train, y_test = y_data_od.iloc[train_index], y_data_od.iloc[test_index]

    # scaling
    scaler = preprocessing.MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # over-sampling
    # print('before', y_train.groupby(['SurvivalWeeks']).size())
    sm = over_sampling.SVMSMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    # print('after', y_train.groupby(['SurvivalWeeks']).size())

    # DataLoader
    train_xt = torch.from_numpy(X_train.astype(np.float32)).cuda(device)
    train_yt = torch.from_numpy(y_train.astype(np.float32)).cuda(device)
    train_data = Data.TensorDataset(train_xt, train_yt)
    train_loader = Data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    print('d')










    # define the model
#     model = ExtraTreesClassifier(n_estimators=250,  random_state=42)
#     model.fit(X_train, y_train.values.ravel())
#     y_pred = model.predict_proba(X_test)
#     fpr, tpr, thresholds = roc_curve(y_test, y_pred[:, 1])
#     auroc = auc(fpr, tpr)
#     print('auc', auroc)
#     all_auroc.append(auroc)
# print(np.mean(all_auroc), np.std(all_auroc))
